In [34]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, OpenAIChatCompletionsModel
from openai import AsyncOpenAI

In [35]:
load_dotenv()

True

In [36]:
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    raise ValueError("GEMINI_API_KEY is not set in environment variables.")

In [37]:
client = AsyncOpenAI(
    api_key=api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)


In [38]:
model = OpenAIChatCompletionsModel(
        model="gemini-2.0-flash",
        openai_client=client
    )

In [39]:
agent: Agent = Agent(name="Assistant", instructions="You are an Assistant Agent of a Bank who will respond to queries only related to ATM.", model=model)

In [40]:
result = await Runner.run(agent, "Guide me")

In [41]:
result.final_output

'Okay, I can help with that! To guide you best, please tell me what you need help with regarding ATMs. For example, are you looking for:\n\n*   **How to find an ATM?**\n*   **How to use an ATM?**\n*   **What to do if your ATM card is lost or stolen?**\n*   **Information on ATM fees?**\n*   **Help with a specific ATM transaction issue?**\n\nThe more information you give me, the better I can assist you.\n'

In [42]:
from pydantic import BaseModel
from agents import Agent

In [43]:
class Recipe(BaseModel):
    title: str
    ingredient: list[str]
    cooking_time: int
    servings: int

In [44]:
recipe_agent: Agent = Agent(name="Recipe Agent", instructions="You are an agent for creating recipes. You will be given the name of a food and your job is to output that as an actual detailed recipe. The cooking time should be in minutes.", model=model, output_type=Recipe)

In [45]:
response = await Runner.run(recipe_agent, "Chocolate Cake")

In [46]:
response.final_output

Recipe(title='Chocolate Cake', ingredient=['2 cups all-purpose flour', '2 cups sugar', '¾ cup unsweetened cocoa powder', '1 ½ teaspoons baking powder', '1 ½ teaspoons baking soda', '1 teaspoon salt', '1 cup buttermilk', '½ cup vegetable oil', '2 large eggs', '2 teaspoons vanilla extract', '1 cup boiling water'], cooking_time=50, servings=12)

In [47]:
from agents import Agent, function_tool

In [48]:
@function_tool
def get_weather(city: str) -> str:
    print(f"Getting weather for {city}.")
    return f"The weather in {city} is hot."

In [49]:
weather_agent: Agent = Agent(name="Weather Agent", instructions="You are the local weather agent. You are given a city and you need to tell the weather and temperature. For any unrelated queries, say I cant help with that.", model=model, tools=[get_weather])

In [50]:
weather_result = await Runner.run(weather_agent, "Dallas")

Getting weather for Dallas.


In [51]:
weather_result.final_output

'The weather in Dallas is hot.\n'

In [52]:
from agents import WebSearchTool

In [56]:
news_agent = Agent(
    name="News Reporter",
    instructions="You are a news reporter. Your job is to find recent news articles on the internet about US politics.",
    model=model,
    tools=[WebSearchTool()]
)

In [57]:
news_result = await Runner.run(news_agent, "find news")

UserError: Hosted tools are not supported with the ChatCompletions API. Got tool type: <class 'agents.tool.WebSearchTool'>, tool: WebSearchTool(user_location=None, search_context_size='medium')

In [ ]:
news_agent.final_output

In [59]:
class Tutorial(BaseModel):
    outline: str
    tutorial: str

In [62]:
tutorial_generator: Agent = Agent(name="Tutorial Generator", instructions=(
        "Given a programming topic and an outline, your job is to generate code snippets for each section of the outline."
        "Format the tutorial in Markdown using a mix of text for explanation and code snippets for examples."
        "Where it makes sense, include comments in the code snippets to further explain the code."
    ),
    handoff_description="Used for generating a tutorial based on an outline.",
    output_type=Tutorial,
    model=model
                                  
                                  )

In [63]:
outline_builder: Agent = Agent(
    name="Outline Builder",
    instructions=(
        "Given a particular programming topic, your job is to help come up with a tutorial. You will do that by crafting an outline."
        "After making the outline, hand it to the tutorial generator agent."
    ),
handoffs=[tutorial_generator],
model=model
)

In [64]:
tutorial_result = await Runner.run(outline_builder, "Loops in Python")

In [65]:
tutorial_result.final_output

Tutorial(outline='1.  **Introduction to Loops**\n    *   What are loops and why are they used?\n    *   Types of loops in Python: `for` and `while`\n\n2.  **`for` Loops**\n    *   Basic syntax of a `for` loop\n    *   Iterating through a sequence (list, tuple, string)\n    *   Using `range()` with `for` loops\n    *   `for` loops with `else`\n    *   Nested `for` loops\n\n3.  **`while` Loops**\n    *   Basic syntax of a `while` loop\n    *   Using `while` loops with conditional statements\n    *   `while` loops with `else`\n    *   Infinite loops and how to avoid them\n\n4.  **Loop Control Statements**\n    *   `break` statement: Exiting a loop\n    *   `continue` statement: Skipping to the next iteration\n    *   `pass` statement: Doing nothing\n\n5.  **Examples and Use Cases**\n    *   Looping through lists and performing operations\n    *   Reading data from a file line by line\n    *   Creating simple games or simulations\n\n6.  **Best Practices and Common Errors**\n    *   Avoidin